# Pruning at initialization with Grasp

In [1]:
import sys
sys.path.append('..')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils.utils import progress_bar, train, test
from utils.pruner import pruning_model_random, check_sparsity, pruning_model, grasp_pruning

In [2]:
lr = 0.1
pruning_ratio = 0.99
epochs = 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [3]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Model
print('==> Building model..')
net = ResNet18()
net = net.to(device)

# Prune
grasp_pruning(net, pruning_ratio, trainloader, num_class=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                    momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

for epoch in range(start_epoch, start_epoch+epochs):
    train(epoch, device, net, trainloader, optimizer, criterion)
    test(epoch, device, net, testloader, criterion, best_acc)
    scheduler.step()
check_sparsity(net)

print("Finished!")

==> Preparing data..


0.6%

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
==> Building model..
start grasp pruning

Epoch: 0
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.582 | Acc: 41.134% (20567/5000 391/391 
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.445 | Acc: 48.300% (4830/1000 100/100 ================>......]  Step: 0ms | Tot: 0ms | Loss: 1.439 | Acc: 48.421% (3680/760 76/100 
Saving..
* remain weight =  0.9999971324191503 %
Finished!
